In [5]:
from langchain_core.documents import Document
from langchain.document_loaders import PyPDFLoader, DirectoryLoader, PyMuPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [ ]:
dir_loader = DirectoryLoader("../data/pdf_files", glob="*.pdf", loader_cls=PyMuPDFLoader)
documents = dir_loader.load()
documents

[Document(metadata={'producer': 'pdfTeX-1.40.26', 'creator': 'TeX', 'creationdate': '2025-10-06T20:11:43+00:00', 'source': '../data/pdf_files/Cover_Letter (1).pdf', 'file_path': '../data/pdf_files/Cover_Letter (1).pdf', 'total_pages': 1, 'format': 'PDF 1.5', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'moddate': '2025-10-06T20:11:43+00:00', 'trapped': '', 'modDate': 'D:20251006201143Z', 'creationDate': 'D:20251006201143Z', 'page': 0}, page_content='Shree Gopalakrishnan\n858-761-2827\n—\nshree.hridai@gmail.com\n—\nlinkedin.com/in/shree-gk\n—\ngithub.com/Shree-G\nOctober 6, 2025\nVP of Engineering\nSupplyframe, a Siemens Company\nPasadena, CA\nDear VP of Engineering,\nI was genuinely excited to see the opening for a Full Stack Engineer at Supplyframe. Your mission to build\ntools that change how engineers research, design, and create new hardware immediately caught my attention.\nI’ve always been driven by the process of taking a great idea from a rough concept all the way 

In [10]:
# Text Splitting

def split_documents(documents, chunk_size=500, chunk_overlap=50):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        length_function=len,
        separators=["\n\n", "\n", " ", ""]

    )
    return text_splitter.split_documents(documents)

split_docs = split_documents(documents)
split_docs

[Document(metadata={'producer': 'pdfTeX-1.40.26', 'creator': 'TeX', 'creationdate': '2025-10-06T20:11:43+00:00', 'source': '../data/pdf_files/Cover_Letter (1).pdf', 'file_path': '../data/pdf_files/Cover_Letter (1).pdf', 'total_pages': 1, 'format': 'PDF 1.5', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'moddate': '2025-10-06T20:11:43+00:00', 'trapped': '', 'modDate': 'D:20251006201143Z', 'creationDate': 'D:20251006201143Z', 'page': 0}, page_content='Shree Gopalakrishnan\n858-761-2827\n—\nshree.hridai@gmail.com\n—\nlinkedin.com/in/shree-gk\n—\ngithub.com/Shree-G\nOctober 6, 2025\nVP of Engineering\nSupplyframe, a Siemens Company\nPasadena, CA\nDear VP of Engineering,\nI was genuinely excited to see the opening for a Full Stack Engineer at Supplyframe. Your mission to build\ntools that change how engineers research, design, and create new hardware immediately caught my attention.'),
 Document(metadata={'producer': 'pdfTeX-1.40.26', 'creator': 'TeX', 'creationdate': '2025-10-

In [13]:
import numpy as np
import os
from sentence_transformers import SentenceTransformer
import chromadb
from chromadb.config import Settings
import uuid
from typing import List, Dict, Any, Tuple
from sklearn.metrics.pairwise import cosine_similarity

In [9]:
class EmbeddingManager:
    def __init__(self, model_name: str = "all-MiniLM-L6-v2"):
        self.model_name = model_name
        self.model = None
        self._load_model() # calls the private method to load the model

    def _load_model(self):
        try:
            self.model = SentenceTransformer(self.model_name)
            print(f"Embedding dimension: {self.model.get_sentence_embedding_dimension()}")
        except Exception as e:
            print(f"Error loading model: {self.model_name}. Error: {e}")
            raise

    def generate_embeddings(self, texts : list[str]) -> np.ndarray:
        if not self.model:
            raise ValueError("Model not selected.")

        embeddings = self.model.encode(texts, show_progress_bar=True)
        print(f"shape of embeddings {embeddings.shape}")
        return embeddings

In [11]:
embedding_manager = EmbeddingManager()
embedding_manager

Embedding dimension: 384


In [15]:
class VectorStore:
    """Manages document embeddings in a ChromaDB vector store"""
    
    def __init__(self, collection_name: str = "pdf_documents", persist_directory: str = "../data/vector_store"):
        """
        Initialize the vector store
        
        Args:
            collection_name: Name of the ChromaDB collection
            persist_directory: Directory to persist the vector store
        """
        self.collection_name = collection_name
        self.persist_directory = persist_directory
        self.client = None
        self.collection = None
        self._initialize_store()

    def _initialize_store(self):
        """Initialize ChromaDB client and collection"""
        try:
            # Create persistent ChromaDB client
            os.makedirs(self.persist_directory, exist_ok=True)
            self.client = chromadb.PersistentClient(path=self.persist_directory)
            
            # Get or create collection
            self.collection = self.client.get_or_create_collection(
                name=self.collection_name,
                metadata={"description": "PDF document embeddings for RAG"}
            )
            print(f"Vector store initialized. Collection: {self.collection_name}")
            print(f"Existing documents in collection: {self.collection.count()}")
            
        except Exception as e:
            print(f"Error initializing vector store: {e}")
            raise

    def add_documents(self, documents: List[Any], embeddings: np.ndarray):
        """
        Add documents and their embeddings to the vector store
        
        Args:
            documents: List of LangChain documents
            embeddings: Corresponding embeddings for the documents
        """
        if len(documents) != len(embeddings):
            raise ValueError("Number of documents must match number of embeddings")
        
        print(f"Adding {len(documents)} documents to vector store...")
        
        # Prepare data for ChromaDB
        ids = []
        metadatas = []
        documents_text = []
        embeddings_list = []
        
        for i, (doc, embedding) in enumerate(zip(documents, embeddings)):
            # Generate unique ID
            doc_id = f"doc_{uuid.uuid4().hex[:8]}_{i}"
            ids.append(doc_id)
            
            # Prepare metadata
            metadata = dict(doc.metadata)
            metadata['doc_index'] = i
            metadata['content_length'] = len(doc.page_content)
            metadatas.append(metadata)
            
            # Document content
            documents_text.append(doc.page_content)
            
            # Embedding
            embeddings_list.append(embedding.tolist())
        
        # Add to collection
        try:
            self.collection.add(
                ids=ids,
                embeddings=embeddings_list,
                metadatas=metadatas,
                documents=documents_text
            )
            print(f"Successfully added {len(documents)} documents to vector store")
            print(f"Total documents in collection: {self.collection.count()}")
            
        except Exception as e:
            print(f"Error adding documents to vector store: {e}")
            raise

vectorstore=VectorStore()
vectorstore

Vector store initialized. Collection: pdf_documents
Existing documents in collection: 0


In [22]:
chunks = split_docs

In [23]:
## Converting the text to embeddings

texts = [chunk.page_content for chunk in chunks]

embeddings = embedding_manager.generate_embeddings(texts)
vectorstore.add_documents(chunks, embeddings)

Batches: 100%|██████████| 1/1 [00:02<00:00,  2.18s/it]

shape of embeddings (31, 384)
Adding 31 documents to vector store...
Successfully added 31 documents to vector store
Total documents in collection: 31
